## RANDOM FOREST

Random forest is an classification and regression algorithm which uses a lot of decision trees. It uses bagging and feature randomness when building each of the tree to try to create the uncorrelated forest<br>It sloves tha major problem of decision tree that is overfitting. Decision trees are not that flexible whereas random forest has simplicity of decision tree and flexibilty which results in vast improvement in accuracy.<br>
In random forrest many trees are build on boorsatrapped dataset.<br>
To create bootstrapped data that is same size as original we just perform the random sampling technique with replacement
<img src="randomf1.png" width="500" height="100"/>

Now after Creating the bootstrapped samples/datasets we will build a tree on each data such that the trees are not correlated .<br>
The major difference between bootstrap aggregation and random forest is in bootstrap aggregation we grow trees on bootstrapped samples by considering all the features at each level Whereas in Rnadom forest we do not consider all the features instead of that at each node/level we consider different type of features that helps in decorrelating the tree and also helps in increasing the accuracy by reducing the overfitting

## The Tasks We are going to do will be
1. Prepare and Load the data
2. Divide the data into training data and testing data
3. create Helper Functions whcih are helpful for our random forest algorithm.<br>
    3.1. BOOTSTRAP SAMPLES--- We will make a function that returns a bootstrapped samples so that we can build our tree.<br>
    3.2. BASE CONDITION---First we will check the node Purity it means if the examples present in the node only contains    single unique label and the other base condition for pruning the tree will be maximum depth if the tree will reach maxium depth then also we will stop their and peform classification.<br>
    3.3. IF the node is not pure or it is not at pruning level so we need to split the node but on the basis of what parameters we should split it for that.
       3.3.a Find POTENTIAL SPLITS.
       3.3.b Among the potential splits DETERMINE THE BEST SPLIT.
       3.3.c SPLIT the data.
       3.3.d To determine the best split take the splitted data and calculate OVERLL ENTROPY.
       3.3.e To calculate overall entropy first calculate ENTROPY.
    3.4 Grow trees on each bootstrap sample which is done using RANDOM_FOREST_ALGO.<br>
    3.5 A function which predict accuracy for each example on all tress is made and applied to whole data frame.<br>
    3.6 For a single example we get lot of predictions a new datframe of predicitions is made in which every column represent output from that tree so will will chose the label which has high frequency 

## Loading and Preparing the data


In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
red_wine = pd.read_csv("winequality_red.csv")
white_wine = pd.read_csv("winequality_white.csv")

In [3]:
red_wine.shape


(1599, 12)

In [4]:
red_wine["colour"]="red"
white_wine["colour"]="white"

In [5]:
red_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,colour
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red


In [6]:
white_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,colour
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white


In [7]:
data = pd.concat([red_wine,white_wine])

In [8]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,colour
0,7.4,0.700,0.00,1.90,0.076,11.0,34.0,0.99780,3.51,0.56,9.400000,5,red
1,7.8,0.880,0.00,2.60,0.098,25.0,67.0,0.99680,3.20,0.68,9.800000,5,red
2,7.8,0.760,0.04,2.30,0.092,15.0,54.0,0.99700,3.26,0.65,9.800000,5,red
3,11.2,0.280,0.56,1.90,0.075,17.0,60.0,0.99800,3.16,0.58,9.800000,6,red
4,7.4,0.700,0.00,1.90,0.076,11.0,34.0,0.99780,3.51,0.56,9.400000,5,red
5,7.4,0.660,0.00,1.80,0.075,13.0,40.0,0.99780,3.51,0.56,9.400000,5,red
6,7.9,0.600,0.06,1.60,0.069,15.0,59.0,0.99640,3.30,0.46,9.400000,5,red
7,7.3,0.650,0.00,1.20,0.065,15.0,21.0,0.99460,3.39,0.47,10.000000,7,red
8,7.8,0.580,0.02,2.00,0.073,9.0,18.0,0.99680,3.36,0.57,9.500000,7,red
9,7.5,0.500,0.36,6.10,0.071,17.0,102.0,0.99780,3.35,0.80,10.500000,5,red


In [9]:
#shuffling a dataset
data_array = np.array(data)
np.random.shuffle(data_array)
data = pd.DataFrame(data_array,columns=data.columns)

In [10]:
data


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,colour
0,6.7,0.24,0.35,13.1,0.05,64,205,0.997,3.15,0.5,9.5,5,white
1,6.5,0.23,0.36,16.3,0.038,43,133,0.99924,3.26,0.41,8.8,5,white
2,6.8,0.26,0.44,8.2,0.046,52,183,0.99584,3.2,0.51,9.4,5,white
3,7.2,0.24,0.28,1.9,0.032,30,92,0.9914,3.1,0.39,10.9,6,white
4,6.9,0.49,0.24,1.2,0.049,13,125,0.9932,3.17,0.51,9.4,5,white
5,9.9,0.49,0.23,2.4,0.087,19,115,0.9948,2.77,0.44,9.4,6,white
6,8.8,0.33,0.41,5.9,0.073,7,13,0.99658,3.3,0.62,12.1,7,red
7,6.8,0.39,0.34,7.4,0.02,38,133,0.99212,3.18,0.44,12,7,white
8,5.9,0.29,0.28,3.2,0.035,16,117,0.98959,3.26,0.42,12.6,6,white
9,6.7,0.46,0.27,5.2,0.039,35,96,0.99129,3.16,0.44,12.4,7,white


### Checking if the data has missing values or not

In [11]:
labels = data["quality"]
del data["quality"]
data["labels"] = labels

In [12]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,colour,labels
0,6.7,0.24,0.35,13.1,0.05,64,205,0.997,3.15,0.5,9.5,white,5
1,6.5,0.23,0.36,16.3,0.038,43,133,0.99924,3.26,0.41,8.8,white,5
2,6.8,0.26,0.44,8.2,0.046,52,183,0.99584,3.2,0.51,9.4,white,5
3,7.2,0.24,0.28,1.9,0.032,30,92,0.9914,3.1,0.39,10.9,white,6
4,6.9,0.49,0.24,1.2,0.049,13,125,0.9932,3.17,0.51,9.4,white,5


In [13]:
def fill_missing_values(column):
    mean = data[column].mean()
    data["column"].fillna(value = mean , inplace =True)

In [14]:
for column in data.columns:
    if column != "quality":
        null = data[column].isnull().values.any()
        if null:
            fill_missing_vales(column)
            print("Missing values has been filled")

In [15]:
d1 = data.iloc[:,:-2]
d1 = (d1-d1.mean())/d1.std()
d1["colour"]=data["colour"]
d1["labels"]=data["labels"]
data=d1

In [16]:
data["colour"].replace(to_replace =["red","white"],value = [1,0],inplace=True )
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,colour,labels
0,-0.39748,-0.60537,0.215849,1.60931,-0.172231,1.88596,1.57913,0.768128,-0.426034,-0.210128,-0.831551,0,5
1,-0.55175,-0.66611,0.284664,2.28189,-0.514759,0.702823,0.305288,1.51513,0.2581,-0.814943,-1.41845,0,5
2,-0.320346,-0.48389,0.835181,0.57942,-0.286407,1.20988,1.1899,0.381291,-0.115064,-0.142926,-0.915394,0,5
3,-0.0118071,-0.60537,-0.265853,-0.744721,-0.686023,-0.0295965,-0.420095,-1.09936,-0.737004,-0.949346,0.342245,0,6
4,-0.243211,0.913127,-0.541112,-0.891847,-0.200775,-0.987375,0.16375,-0.499099,-0.301646,-0.142926,-0.915394,0,5


## Train Test Split

In [17]:
def transform_label(value):
    if value <= 5:
        return "bad"
    else:
        return "good"

data["labels"] = data.labels.apply(transform_label)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,colour,labels
0,-0.39748,-0.60537,0.215849,1.60931,-0.172231,1.88596,1.57913,0.768128,-0.426034,-0.210128,-0.831551,0,bad
1,-0.55175,-0.66611,0.284664,2.28189,-0.514759,0.702823,0.305288,1.51513,0.2581,-0.814943,-1.41845,0,bad
2,-0.320346,-0.48389,0.835181,0.57942,-0.286407,1.20988,1.1899,0.381291,-0.115064,-0.142926,-0.915394,0,bad
3,-0.0118071,-0.60537,-0.265853,-0.744721,-0.686023,-0.0295965,-0.420095,-1.09936,-0.737004,-0.949346,0.342245,0,good
4,-0.243211,0.913127,-0.541112,-0.891847,-0.200775,-0.987375,0.16375,-0.499099,-0.301646,-0.142926,-0.915394,0,bad
5,2.07083,0.913127,-0.609926,-0.63963,0.883898,-0.649336,-0.0131732,0.0344706,-2.78941,-0.613338,-0.915394,0,good
6,1.22235,-0.0587112,0.628737,0.0960033,0.484282,-1.32541,-1.81778,0.628067,0.506876,0.596292,1.34836,1,good
7,-0.320346,0.305728,0.147035,0.411275,-1.02855,0.421123,0.305288,-0.859258,-0.239452,-0.613338,1.26451,0,good
8,-1.01456,-0.301671,-0.265853,-0.471485,-0.600391,-0.818356,0.0222113,-1.70296,0.2581,-0.747741,1.76757,0,good
9,-0.39748,0.730907,-0.334668,-0.0511234,-0.486215,0.252103,-0.349326,-1.13605,-0.36384,-0.613338,1.59988,0,good


In [18]:
def train_test_split(data,train_per):
    train_df = data.iloc[:int(train_per*len(data)),:]
    test_df = data.iloc[int(train_per*len(data)):,:]
    return train_df,test_df

In [19]:
train_df,test_df = train_test_split(data,train_per=0.80)

In [20]:
print("dimesnions of training data and testing data",train_df.shape,test_df.shape)

dimesnions of training data and testing data (5197, 13) (1300, 13)


## Helper Functions


In [21]:
np.unique(data["colour"])

array([0, 1], dtype=int64)

In [22]:
def determine_feature_type(data):
    
    feature_type={}
     
    for column in data.columns[:-1]:
        unq_val = np.unique(data[column])
        ex_0 = unq_val[0]
        if type(ex_0) == str or len(unq_val)>12:
                
            feature_type[column]="continious"
                
        else:
                
            feature_type[column]="categorical"
                
    return feature_type

In [23]:
def Bootstrap(data,n_bootstap):
    
    samples_index = np.random.randint(low=0 ,high=len(data),size=n_bootstap)
    
    bootstrap_data = data.iloc[samples_index]
    
    return bootstrap_data
    

In [24]:
def check_purity(data):
    
    label = data[:,-1]
    
    unq_labels = np.unique(label)
    
    if len(unq_labels) == 1:
    
        return True
    
    else:
    
        return False

In [25]:
def classify(data):
    
    label = data[:,-1]
    
    unq_lbl , count = np.unique(label,return_counts=True)
    
    max_freq_index = np.argmax(count)
    
    classification_val = unq_lbl[max_freq_index]   # return the max freq label
    
    return  classification_val

In [26]:
def get_potential_splits(data,random_subspace):   #here random_subspace is no to feature we are taking in account
    
    potential_splits={}
    
    column_indices = range(data.shape[1]-1) 
    
    if random_subspace and random_subspace<=len(column_indices):
        
        feature_subset = random.sample(population=column_indices,k=random_subspace)
    
    for col in feature_subset :   # traversing through each column
        
        unq_values = np.unique(data[:,col])  # finding unique value of that particular column
        
        potential_splits[col] = unq_values   #adding that into potential_splits
                
    return potential_splits

def get_potential_splits(data,random_subspace):   #here random_subspace is no to feature we are taking in account
    
    potential_splits={}
    
    column_indices = range(data.shape[1]-1) 
    
    if random_subspace and random_subspace<=len(column_indices):
        
        feature_subset = random.sample(population=column_indices,k=random_subspace)
    
    for col in feature_subset :   # traversing through each column
            unique_col_values = np.unique(data[:,col])
            feature_type_key = labels[col]
            type_of_feature = feature_type[feature_type_key]
            #print(column)
            if type_of_feature == "continious":
                potential_splits[column]=[]
                for val in range(len(unique_col_values)):  # this loop is used to take the sum of unq values and take thier mean so that its easier to split them

                    if val !=0:

                        previous_val = unique_col_values[val-1]

                        current_val = unique_col_values[val]

                        potential_splits[column].append((current_val+previous_val)/2)

            else:
                potential_splits[column]=unique_col_values
    return potential_splits

In [27]:
def split_data(data,split_col,split_val):
    split_col_values = data[:,split_col]   # columns -- split col values
    
    split_feature = labels[split_col]
    type_of_feature = feature_type[split_feature]
    
    if type_of_feature=="continious":
        data_above = data[split_col_values  > split_val]
        data_below = data[split_col_values  <= split_val]
    else:
        data_below = data[split_col_values  == split_val]
        data_above = data[split_col_values  != split_val]
    return data_above,data_below

In [28]:
def Entropy(data):
    _,count = np.unique(data[:,-1],return_counts=True)  #counts conatins occurace of each class
    probabilities = count/np.sum(count)   # prob of each class
    entropy = sum(probabilities*(-np.log2(probabilities)))   # entropies
    
    return entropy

In [29]:
def get_overall_entropy(data_above,data_below):
    entropy_above = Entropy(data_above)
    entropy_below = Entropy(data_below)
    n = len(data_above)+len(data_below)
    prob_above = len(data_above)/n
    prob_below = len(data_below)/n
    overall_entropy = prob_above*entropy_above+prob_below*entropy_below
    return overall_entropy

In [31]:
def determine_best_split(data,potential_splits):
    overall_entropy = 9999
    for split_col in potential_splits:
    
        for split_val in (potential_splits[split_col]):
            
            data_above , data_below = split_data(data , split_col,split_val)
            
            calculated_overall_entropy = get_overall_entropy(data_above,data_below)
            
            if calculated_overall_entropy <= overall_entropy:
                
                overall_entropy = calculated_overall_entropy
                
                best_split_col = split_col
                
                best_split_val = split_val
                
    return best_split_col,best_split_val

## Decision Tree Algorithm to implement all the helper functions


In [44]:
def decision_tree_algo(df,counter=0,random_subspace=None,max_depth = 12): 
    if counter == 0:     #if counter =0 means 1st iteration so df is data frame type and we need to convert it in array
        global labels
        global feature_type
        labels = df.columns
        feature_type = determine_feature_type(df)
        data=df.values                                     # converting it to an array
    else:
        data = df
        
    # checking the base condition
    if check_purity(data): #or max_depth==counter:
        classification = classify(data)
        return classification
    
    else:
        
        counter+=1
        potential_splits = get_potential_splits(data,random_subspace)
        best_split_col,best_split_val = determine_best_split(data,potential_splits)
        data_above,data_below = split_data(data,best_split_col,best_split_val)
        
        best_split_feature = labels[best_split_col]   # name of best split feature to make the question
        type_of_feature = feature_type[best_split_feature]
        #print(best_split_feature,best_split_val)
        if type_of_feature == "continious":
            question = str(best_split_feature)+" <= " +str(best_split_val)
        else:
            question = str(best_split_feature)+" = " +str(best_split_val)
            
        sub_tree={question:[]}
        
        yes_ans = decision_tree_algo(data_below,counter,random_subspace,max_depth)
        no_ans = decision_tree_algo(data_above,counter,random_subspace,max_depth)
        
        if yes_ans==no_ans:
            sub_tree[question].append(yes_ans)
            sub_tree[question].append(no_ans)
        else:
            sub_tree[question].append(yes_ans)
            sub_tree[question].append(no_ans)
            
        return sub_tree
    

## Random Forest Algorithm

In [49]:
def random_forest_algorithm(train_df,n_tree,n_bootstrap,n_feature,max_depth):   # n_bootstrap is size of bootstrap sample
    forest = []  # to append alll the trees
    for sample in range(n_tree):
        bootstrap_sample = Bootstrap(data,n_bootstrap)
        #print(bootstrap_sample)
        tree = decision_tree_algo(bootstrap_sample,random_subspace=n_feature,max_depth=max_depth)
        #print(tree)
        forest.append(tree)
    return forest

In [50]:
random_forest = random_forest_algorithm(train_df,n_tree=3,n_bootstrap=800,n_feature=4,max_depth = 4)

In [51]:
random_forest

[{'density <= -0.7258656840173487': [{'volatile acidity <= -1.0305492791646031': [{'citric acid <= -0.1970385083095257': [{'alcohol <= -0.24465327135931397': ['bad',
         'good']},
       'good']},
     {'residual sugar <= -0.597594077620886': [{'free sulfur dioxide <= -0.5366558588194927': [{'sulphates <= 0.6634934491172625': [{'residual sugar <= -0.91286559384524': ['bad',
             {'volatile acidity <= 0.002028713926790104': [{'free sulfur dioxide <= -1.0437152588898189': ['good',
                 {'free sulfur dioxide <= -0.9873753255486715': ['bad',
                   {'pH <= -0.5504222064099913': ['good',
                     {'free sulfur dioxide <= -0.8746954588663768': ['good',
                       {'sulphates <= 0.327485207066719': [{'citric acid <= -0.9539998088180239': ['bad',
                           {'pH <= 0.13371184506046474': [{'citric acid <= -0.05940918094434406': ['bad',
                               {'sulphates <= -0.7477411674950191': ['bad',
        

In [52]:
def classify_example(example,tree):     # example here depicts 1 single example
    
    question = list(tree.keys())[0]     # 1st question
    #print(question.split())
    splitted_ques = question.split()
    # as the list has been didvided  into multiple parts as it breaks it in space and single feature col has more than one word
    if len(splitted_ques)==4:
        split_feature=splitted_ques[0]+" "+splitted_ques[1]
        comparision_operator = splitted_ques[2]
        split_val = splitted_ques[3]
    elif len(splitted_ques)==5:
        split_feature = splitted_ques[0]+" "+splitted_ques[1]+" "+splitted_ques[2]
        comparision_operator = splitted_ques[3]
        split_val = splitted_ques[4]
    else:
        split_feature, comparision_operator,split_val = question.split()
    if comparision_operator == "<=":
        if example[split_feature] <= float(split_val):
            answer = tree[question][0]                  # if example feature value is less then it goes to true ans in list

        else:
            #print(tree[question])
            #print(example[split_feature])
            answer= tree[question][1]                   # else 1 index is for false answer
        
        
    else:
        if str(example[split_feature]) == split_val:
            
            answer = tree[question][0]                  # if example feature value is less then it goes to true ans in list
        
        else:

            answer= tree[question][1]   
        
    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [53]:
def decision_tree_predictions(test_df,tree):
    tree_prediction = test_df.apply(classify_example, axis=1, args=(tree,))
    return tree_prediction

In [54]:
def Random_forest_predictions(df,random_forest):
    predictions={}
    
    for i in range(len(random_forest)):
    
        column_name = "tree_"+str(i)
        
        predictions[column_name]= decision_tree_predictions(test_df,random_forest[i])
        
    prediction_Dataframe = pd.DataFrame(predictions)
    final_predictions=prediction_Dataframe.mode(axis=1)[0]
    
    return final_predictions      

In [55]:
def cal_accuracy(test_df,random_forest):
    calculated_predictions = Random_forest_predictions(test_df,random_forest)
    actual_values = test_df["labels"]
    count = np.count_nonzero(calculated_predictions == actual_values)
    #print(count)
    accuracy = count/len(test_df)
    return accuracy

In [56]:
accuracy = cal_accuracy(test_df,random_forest)
print("ACCURACY",accuracy)

ACCURACY 0.76
